In [18]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import DBSCAN
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import scale
from datetime import datetime
from IPython.display import display

In [19]:
file_path = 'med_events.csv'
med_events = pd.read_csv(file_path)
med_events['DATE'] = pd.to_datetime(med_events['DATE'], errors='coerce').dt.date
tidy = med_events.copy()

In [20]:
def SeeDBSCAN(arg1, tidy):
    data_med = tidy[tidy['CATEGORY'] == arg1].copy()
    if data_med.empty:
        raise ValueError(f"No data found for medication {arg1}")
        
    data_med = data_med.sort_values(by=['PATIENT_ID', 'DATE'])
    
    intervals = []
    patient_data_list = []
    for patient_id in data_med['PATIENT_ID'].unique():
        patient_data = data_med[data_med['PATIENT_ID'] == patient_id]
        if len(patient_data) > 1:
            dates = pd.to_datetime(patient_data['DATE'])
            patient_intervals = np.diff(dates).astype('timedelta64[D]').astype(int)
            intervals.extend(patient_intervals)
            patient_data_list.append(patient_data)
            
    if len(intervals) == 0:
        raise ValueError("No intervals found. Each patient must have at least two prescriptions.")
    
    ecdf_x = np.sort(intervals)
    ecdf_y = np.arange(1, len(ecdf_x)+1) / len(ecdf_x)
    dfper = pd.DataFrame({'x': ecdf_x, 'y': ecdf_y})
    dfper = dfper[dfper['y'] <= 0.8]
    if len(dfper) == 0:
        raise ValueError("No data points remain after filtering for 80% ECDF")
    
    final_df = pd.concat(patient_data_list, ignore_index=True)
    
    return {'data': final_df, 'ecdf_data': dfper, 'ecdf_x': ecdf_x, 'ecdf_y': ecdf_y}

In [21]:
def plot_ecdf(dfper, ecdf_x, ecdf_y):
    if 'dfper' not in globals():
        raise ValueError("Run SeeDBSCAN first to generate dfper.")
    
    plt.figure(figsize=(6,4))
    plt.subplot(1,2,1)
    plt.plot(dfper['x'], dfper['y'], marker=".", linestyle="none", color='navy')
    plt.xlabel("Event Interval (days)", fontsize=9)
    plt.title("80% ECDF", fontsize=10)
    plt.grid(True, linestyle='--', alpha=0.7)
    
    plt.subplot(1,2,2)
    plt.plot(ecdf_x, ecdf_y, marker=".", linestyle="none", color='darkgreen')
    plt.xlabel("Event Interval (days)", fontsize=9)
    plt.title("100% ECDF", fontsize=10)
    plt.grid(True, linestyle='--', alpha=0.7)
    plt.tight_layout()
    plt.show()
    
    plt.figure(figsize=(6,4))
    sns.kdeplot(np.log(dfper['x']), shade=True, color='purple')
    plt.xlabel("Log(Event Interval)", fontsize=9)
    plt.title("Density Plot: Log(Event Interval)", fontsize=10)
    plt.grid(True, linestyle='--', alpha=0.7)
    plt.tight_layout()
    plt.show()

In [22]:
def run_dbscan(dfper):
    a = dfper[['x']].copy()
    a = scale(a)
    dbscan = DBSCAN(eps=1.5, min_samples=5)
    dfper['cluster'] = dbscan.fit_predict(a)
    
    plt.figure(figsize=(6,4))
    sns.scatterplot(data=dfper, x='x', y='y', hue='cluster', palette='deep', s=50)
    plt.xlabel("Event Interval (days)", fontsize=9)
    plt.ylabel("ECDF Value", fontsize=9)
    plt.title("DBSCAN Clustering on ECDF Data", fontsize=10)
    plt.grid(True, linestyle='--', alpha=0.7)
    plt.tight_layout()
    plt.show()
    
    return dfper

In [23]:
def see_assumption(result, med_label):
    if not isinstance(result, dict) or 'data' not in result:
        raise TypeError("Expected a dictionary with a 'data' key from SeeDBSCAN")
    
    data = result['data']
    data = data.sort_values(by=['PATIENT_ID', 'DATE'])
    data['prev_date'] = data.groupby('PATIENT_ID')['DATE'].shift(1)
    data['Duration'] = data.apply(lambda row: (pd.to_datetime(row['DATE']) - 
                                               pd.to_datetime(row['prev_date'])).days 
                                  if pd.notnull(row['prev_date']) else np.nan, axis=1)
    data = data.dropna(subset=['Duration'])
    data['p_number'] = data.groupby('PATIENT_ID').cumcount() + 1
    data['p_number'] = data['p_number'].astype('category')
    
    medians_of_medians = data.groupby('PATIENT_ID')['Duration'].median().reset_index()
    
    plt.figure(figsize=(8,5))
    sns.boxplot(x='p_number', y='Duration', data=data)
    plt.axhline(y=medians_of_medians['Duration'].median(), color='r', linestyle='--', label='Median of Medians')
    plt.title(f"Duration by Prescription Number ({med_label})", fontsize=12)
    plt.legend(fontsize=9)
    plt.tight_layout()
    plt.show()
    
    return data

In [26]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import DBSCAN
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import scale
from datetime import datetime
from IPython.display import display

### Code Block 1: Load Data
# ------------------------------
file_path = 'med_events.csv'
med_events = pd.read_csv(file_path)
med_events['DATE'] = pd.to_datetime(med_events['DATE'], errors='coerce').dt.date
tidy = med_events.copy()

### Code Block 2: DBSCAN Clustering Function
# ------------------------------
def SeeDBSCAN(arg1, tidy):
    data_med = tidy[tidy['CATEGORY'] == arg1].copy()
    if data_med.empty:
        raise ValueError(f"No data found for medication {arg1}")
        
    data_med = data_med.sort_values(by=['PATIENT_ID', 'DATE'])
    
    intervals = []
    patient_data_list = []
    for patient_id in data_med['PATIENT_ID'].unique():
        patient_data = data_med[data_med['PATIENT_ID'] == patient_id]
        if len(patient_data) > 1:
            dates = pd.to_datetime(patient_data['DATE'])
            patient_intervals = np.diff(dates).astype('timedelta64[D]').astype(int)
            intervals.extend(patient_intervals)
            patient_data_list.append(patient_data)
            
    if len(intervals) == 0:
        raise ValueError("No intervals found. Each patient must have at least two prescriptions.")
    
    ecdf_x = np.sort(intervals)
    ecdf_y = np.arange(1, len(ecdf_x)+1) / len(ecdf_x)
    dfper = pd.DataFrame({'x': ecdf_x, 'y': ecdf_y})
    dfper = dfper[dfper['y'] <= 0.8]
    if len(dfper) == 0:
        raise ValueError("No data points remain after filtering for 80% ECDF")
    
    final_df = pd.concat(patient_data_list, ignore_index=True)
    
    return {'data': final_df, 'ecdf_data': dfper, 'ecdf_x': ecdf_x, 'ecdf_y': ecdf_y}

### Code Block 3: ECDF and Density Plots
# ------------------------------
def plot_ecdf(dfper, ecdf_x, ecdf_y):
    if 'dfper' not in globals():
        raise ValueError("Run SeeDBSCAN first to generate dfper.")
    
    plt.figure(figsize=(6,4))
    plt.subplot(1,2,1)
    plt.plot(dfper['x'], dfper['y'], marker=".", linestyle="none", color='navy')
    plt.xlabel("Event Interval (days)", fontsize=9)
    plt.title("80% ECDF", fontsize=10)
    plt.grid(True, linestyle='--', alpha=0.7)
    
    plt.subplot(1,2,2)
    plt.plot(ecdf_x, ecdf_y, marker=".", linestyle="none", color='darkgreen')
    plt.xlabel("Event Interval (days)", fontsize=9)
    plt.title("100% ECDF", fontsize=10)
    plt.grid(True, linestyle='--', alpha=0.7)
    plt.tight_layout()
    plt.show()
    
    plt.figure(figsize=(6,4))
    sns.kdeplot(np.log(dfper['x']), shade=True, color='purple')
    plt.xlabel("Log(Event Interval)", fontsize=9)
    plt.title("Density Plot: Log(Event Interval)", fontsize=10)
    plt.grid(True, linestyle='--', alpha=0.7)
    plt.tight_layout()
    plt.show()

### Code Block 4: DBSCAN Clustering
# ------------------------------
def run_dbscan(dfper):
    a = dfper[['x']].copy()
    a = scale(a)
    dbscan = DBSCAN(eps=1.5, min_samples=5)
    dfper['cluster'] = dbscan.fit_predict(a)
    
    plt.figure(figsize=(6,4))
    sns.scatterplot(data=dfper, x='x', y='y', hue='cluster', palette='deep', s=50)
    plt.xlabel("Event Interval (days)", fontsize=9)
    plt.ylabel("ECDF Value", fontsize=9)
    plt.title("DBSCAN Clustering on ECDF Data", fontsize=10)
    plt.grid(True, linestyle='--', alpha=0.7)
    plt.tight_layout()
    plt.show()
    
    return dfper

### Code Block 5: Assumption Visualization Function
# ------------------------------
def see_assumption(result, med_label):
    if not isinstance(result, dict) or 'data' not in result:
        raise TypeError("Expected a dictionary with a 'data' key from SeeDBSCAN")
    
    data = result['data']
    data = data.sort_values(by=['PATIENT_ID', 'DATE'])
    data['prev_date'] = data.groupby('PATIENT_ID')['DATE'].shift(1)
    data['Duration'] = data.apply(lambda row: (pd.to_datetime(row['DATE']) - 
                                               pd.to_datetime(row['prev_date'])).days 
                                  if pd.notnull(row['prev_date']) else np.nan, axis=1)
    data = data.dropna(subset=['Duration'])
    data['p_number'] = data.groupby('PATIENT_ID').cumcount() + 1
    data['p_number'] = data['p_number'].astype('category')
    
    medians_of_medians = data.groupby('PATIENT_ID')['Duration'].median().reset_index()
    
    plt.figure(figsize=(8,5))
    sns.boxplot(x='p_number', y='Duration', data=data)
    plt.axhline(y=medians_of_medians['Duration'].median(), color='r', linestyle='--', label='Median of Medians')
    plt.title(f"Duration by Prescription Number ({med_label})", fontsize=12)
    plt.legend(fontsize=9)
    plt.tight_layout()
    plt.show()
    
    return data

### Code Block 6: Execution for medA and medB
# ------------------------------
print("=== DBSCAN Clustering Results for medA ===")
medA_results = SeeDBSCAN("medA", tidy)
plot_ecdf(medA_results['ecdf_data'], medA_results['ecdf_x'], medA_results['ecdf_y'])
medA_clustered = run_dbscan(medA_results['ecdf_data'])
see_assumption(medA_results, "medA")

print("\n=== DBSCAN Clustering Results for medB ===")
medB_results = SeeDBSCAN("medB", tidy)
plot_ecdf(medB_results['ecdf_data'], medB_results['ecdf_x'], medB_results['ecdf_y'])
medB_clustered = run_dbscan(medB_results['ecdf_data'])
see_assumption(medB_results, "medB")


=== DBSCAN Clustering Results for medA ===


ValueError: Run SeeDBSCAN first to generate dfper.